In [15]:
import csv,re#,nltk                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
#from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
import numpy as np

In [16]:
def loadData(path, Text=None):
    with open(path, 'r',encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)
        for line in reader:
            (Id, Text, Rating, Verified, Category, Label) = parseReview(line)
            rawData.append((Id, Text, Rating, Verified, Category, Label))

def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    
    for (_, Text, Rating, Verified, Category, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        other_features = toFeatureVector(preProcess(Text))
        other_features.update({'Rating':Rating, 'Verified':Verified, 'Category':Category}) 
        trainData.append((other_features, Label))
    for (_, Text, Rating, Verified, Category, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        other_features = toFeatureVector(preProcess(Text))
        other_features.update({'Rating':Rating, 'Verified':Verified, 'Category':Category}) 
        testData.append((other_features, Label))

In [17]:
# using the 3 additional features of rating,verification and category increased the F Score:0.605206 to 0.731027 by itself

# Question 1

In [18]:
def parseReview(reviewLine):

    Id = reviewLine[0]
    Text = reviewLine[8]
    Label = reviewLine[1]
    Rating = reviewLine[2] 
    Verified = reviewLine[3]
    Category = reviewLine[4]
    
    if reviewLine[1]  == '__label1__':
        reviewLine[1] = 'fake'
    else :
        reviewLine[1] = 'real'
    

   
    return (Id, Text, Rating, Verified, Category, Label)

    

In [19]:
# the rating chosen as maybe relation with fake reviews and the rating given
# if the reviewer was verified to help with classification and improve scores
# the category as also maybe higher fake reviews in specific categories
# using 3 other feature help improve performance from F Score:0.605206 to 


In [20]:
def preProcess(text):
    # word tokenisation
    # separate out words and strings of punctuation into separate white spaced words
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text) #remove
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)
    tokens = re.split(r"\s+",text)#split the regular expression by the white space
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text) #normalisation
    tokens = [t.lower() for t in tokens] 
    #tokens = word_tokenize(text)
    #tokens = [WordNetLemmatizer().lemmatize(t) for t in tokens] 
    #tokens = [nltk.stem.SnowballStemmer('english').stem(t) for t in tokens]   

    return tokens

In [21]:
# using Lemmatizer also slightly reduced the f score from 0.604973 to 0.589483
# using word_tokenize also slightly reduced the f score from 0.604973 to 0.593786
# using stemming also slightly reduced the f score from 0.604973 to 0.592549


# Question 2

In [22]:
featureDict = {} # 
def toFeatureVector(words):     # returns a dictionary where the features as keys, and weights as values
    v = {}
    for w in words:
        try:
            featureDict[w] += 1
        except KeyError:            
            featureDict[w] = 1
        try:
            v[w] += (1.0/len(words))
        except KeyError:
            v[w] = (1.0/len(words))

    return v


In [23]:
# returns a dictionary with the features as keys, and weights as values where the key is not 
#in the dictionary add it and give it a wiehgt of 1, which improved the finalscore from the previous method of 
# just adding to the word to the dictionary with no weight of the actual word
#increase performance by 3.5% from f score of 0.604973 to 0.637559

In [24]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)

# Question 3

In [25]:
def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    # DESCRIBE YOUR METHOD IN WORDS
    for i in range(0,len(dataset),foldSize):
        trainFolds = dataset[i:i+foldSize]
        validationFold = dataset[:i] + dataset[i+foldSize:]
        classifier = trainClassifier(trainFolds)
        truth = [x[1] for x in validationFold]
        pred = predictLabels(validationFold,classifier)
        cv_results.append(precision_recall_fscore_support(truth, pred, average='weighted'))
    
    return cv_results


In [26]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))

def predictLabel(reviewSample, classifier):
    return classifier.classify(toFeatureVector(preProcess(reviewSample)))

In [27]:
# MAIN
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
# loading reviews
# initialize global lists that will be appended to by the methods below
rawData = []          
trainData = []        
testData = []         

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
reviewPath = 'amazon_reviews.txt'

# Do the actual stuff (i.e. call the functions we've made)
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')
loadData(reviewPath) 

# We split the raw dataset into a set of training data and a set of test data (80/20)
# You do the cross validation on the 80% (training data)
# We print the number of training samples and the number of features before the split
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')
splitData(0.8)
# We print the number of training samples and the number of features after the split
print("After split, %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

# QUESTION 3 - Make sure there is a function call here to the

crossValidate(trainData, 10) # perform 10 folds using the crossvalidate method

Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 21000 rawData, 0 trainData, 0 testData
Preparing training and test data...
After split, 21000 rawData, 16800 trainData, 4200 testData
Training Samples: 
16800
Features: 
43810
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...
Training Classifier...


[(0.7795087974829862, 0.7771164021164021, 0.7765663799945197, None),
 (0.7785870906933464, 0.7755291005291005, 0.7749779681222065, None),
 (0.7806103502496021, 0.7783068783068783, 0.777826718585516, None),
 (0.7797017215336897, 0.7775132275132275, 0.7770898471098344, None),
 (0.7832896108145156, 0.7811507936507937, 0.7807456272316534, None),
 (0.7818203780695714, 0.7794973544973545, 0.7790820243711208, None),
 (0.7795680360575035, 0.7771825396825397, 0.7766735395565812, None),
 (0.7834868074544517, 0.781547619047619, 0.7811832913177862, None),
 (0.7799084526372917, 0.7773809523809524, 0.7768792210967216, None),
 (0.7817506525695831, 0.7794973544973545, 0.7790465082190251, None)]

In [28]:
# using dictionary that uses the features as keys, and weights as values with the 3 additional feature increase the F score from 0.604973 from question 3 to 0.800354  

# Evaluate on test set

In [29]:
# Finally, check the accuracy of your classifier by training on all the tranin data
# and testing on the test set
# Will only work once all functions are complete
functions_complete = True  # set to True once you're happy with your methods for cross val
if functions_complete:
    print(testData[0])   # have a look at the first test data instance
    classifier = trainClassifier(trainData)  # train the classifier
    testTrue = [t[1] for t in testData]   # get the ground-truth labels from the data
    testPred = predictLabels(testData, classifier)  # classify the test data to get predicted labels
    finalScores = precision_recall_fscore_support(testTrue, testPred, average='weighted') # evaluate
    print("Done training!")
    print("Precision: %f\nRecall: %f\nF Score:%f" % finalScores[:3])

({'this': 0.08, 'assortment': 0.04, 'is': 0.04, 'really': 0.04, 'hershey': 0.04, "'": 0.04, 's': 0.04, 'at': 0.04, 'their': 0.04, 'best': 0.04, '.': 0.08, 'the': 0.08, 'little': 0.04, 'ones': 0.04, 'are': 0.04, 'always': 0.04, 'excited': 0.04, 'whenever': 0.04, 'holidays': 0.04, 'come': 0.04, 'because': 0.04, 'of': 0.04, 'Rating': '5', 'Verified': 'N', 'Category': 'Grocery'}, '__label1__')
Training Classifier...
Done training!
Precision: 0.806665
Recall: 0.802857
F Score:0.802243


# Questions 4 and 5
Once you're happy with your functions for Questions 1 to 3, it's advisable you make a copy of this notebook to make a new notebook, and then within it adapt and improve all three functions in the ways asked for in questions 4 and 5.